In [2]:
import geopandas as gpd
import pandas as pd
import matplotlib as plt

si = gpd.read_file('./CTPRVN_20220324/ctp_rvn.shp', encoding = 'euc-kr')
si = si.to_crs(epsg=4326)
dong = gpd.read_file("./EMD_20220324/emd.shp", encoding = 'euc-kr')
dong = dong.to_crs(epsg=4326)
si = si[si['CTP_ENG_NM'] == 'Jeju-do']

dropList = []
for i in range(0, len(dong)):
    if(si['geometry'].intersects(dong['geometry'][i]).bool() == False):
        dropList.append(i)

dong = dong.drop(dropList)

east_list = ["구좌읍", "우도면", "표선면", "성산읍"]
west_list = ["한림읍", "한경면", "대정읍"]
north_list = ["애월읍", "조천읍", "해안동", "봉개동", "회천동", "월평동", "영평동", "연동", "노형동", "용강동", "오라이동", "오등동", "아라일동", "아라이동", "화북일동", "화북이동", "도련일동", "도련이동", "삼양일동", "삼양이동", "삼양삼동", "일도이동", "이도이동", "오라일동", "오라이동", "오라삼동", "건입동", "도남동", "용담일동", "용담이동", "용담삼동", "도두일동", "도두이동", "삼도일동", "삼도이동", "일도일동", "이도일동", "이도이동", "이호일동", "이호이동", "외도일동", "외도이동", "내도동", "도평동", "추자면"]
south_list = ["안덕면", "남원읍", "상예동", "하예동", "색달동", "중문동", "대포동", "회수동", "하원동", "도순동", "영남동", "강정동", "법환동", "서호동", "호근동", "서홍동", "동홍동", "서귀동", "토평동", "상효동", "보목동", "하효동", "신효동"]

dong['LOCATION'] = ""
for index, r in dong.iterrows():
    if(r['EMD_KOR_NM'] in east_list):
        dong.loc[index, 'LOCATION'] = "EAST"
    elif(r['EMD_KOR_NM'] in west_list):
        dong.loc[index, 'LOCATION'] = "WEST"
    elif(r['EMD_KOR_NM'] in north_list):
        dong.loc[index, 'LOCATION'] = "NORTH"
    elif(r['EMD_KOR_NM'] in south_list):
        dong.loc[index, 'LOCATION'] = "SOUTH"

continents = dong.dissolve(by='LOCATION', aggfunc='sum')
jeju_east = continents['geometry']['EAST']
jeju_west = continents['geometry']['WEST']
jeju_north = continents['geometry']['NORTH']
jeju_south = continents['geometry']['SOUTH']

In [3]:
import folium
m = folium.Map(location = [33.3734235,126.5512051,], zoom_start=10)
m.save('jeju_map.html')

for _, r in dong.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    if(r['LOCATION'] == "EAST"):
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'green'})
    elif(r['LOCATION'] == "WEST"):
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'blue'})
    elif(r['LOCATION'] == "NORTH"):
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'red'})
    elif(r['LOCATION'] == "SOUTH"):
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'orange'})
    else:
        geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'black'})

    lat = r['geometry'].centroid.y
    lon = r['geometry'].centroid.x
    folium.Marker(location=[lat, lon],
                  popup='{}'.format(r['EMD_KOR_NM'])).add_to(m)


    geo_j.add_to(m)
m